# Player Gesture / Pose Detection

This notebook implements a pipeline to detect human poses in images
and classify player gestures using pre-trained pose estimation
(keypoints) and a lightweight CNN/MLP.

The workflow demonstrates GPU acceleration, reproducible pipelines,
and computer vision skills relevant for game analytics.


In [17]:
import sys, os
sys.path.append(os.path.abspath(".."))

import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from collections import Counter

from src.dataset import PoseDataset
from src.model import PoseMLP
from src.train import train_model
from src.evaluate import evaluate_model
from src.utils import save_model

print("Import Complete")

Import Complete


In [18]:
DATA_IMAGE_DIR = "../data/raw/images/"
ANNOTATION_FILE = "../data/raw/cctvAnnotations.json"

CLASS_NAMES = ["stand", "sit", "raise_hand"]
NUM_CLASSES = len(CLASS_NAMES)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


## Transform

In [19]:
transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

dataset = PoseDataset(
    image_dir=DATA_IMAGE_DIR,
    annotation_file=ANNOTATION_FILE,
    transform=transform
)

print("Dataset size:", len(dataset))


[INFO] Loaded 933 valid samples
Dataset size: 933


In [20]:
indices = list(range(len(dataset)))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

from torch.utils.data import Subset
train_ds = Subset(dataset, train_idx)
val_ds = Subset(dataset, val_idx)

train_labels = [label.item() for _, label in train_ds]
class_counts = Counter(train_labels)
weights = [1.0 / class_counts[label] for label in train_labels]

sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)


## Model

In [21]:
INPUT_DIM = dataset[0][0].shape[0]

model = PoseMLP(
    input_dim=INPUT_DIM,
    num_classes=NUM_CLASSES
)

class_weights = torch.tensor([1.0 / class_counts.get(i,1) for i in range(NUM_CLASSES)], device=device)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


## Training

In [22]:
model = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs=25
)


100%|██████████| 12/12 [00:07<00:00,  1.54it/s]


Epoch [1/25] | Loss: 24.2435 | Val Acc: 0.7647


100%|██████████| 12/12 [00:07<00:00,  1.55it/s]


Epoch [2/25] | Loss: 10.4858 | Val Acc: 0.4706


100%|██████████| 12/12 [00:07<00:00,  1.56it/s]


...

Epoch [23/25] | Loss: 2.1185 | Val Acc: 0.8503


100%|██████████| 12/12 [00:07<00:00,  1.60it/s]


Epoch [24/25] | Loss: 1.6851 | Val Acc: 0.9412


100%|██████████| 12/12 [00:07<00:00,  1.58it/s]


Epoch [25/25] | Loss: 3.2677 | Val Acc: 0.7594


## Evaluation

In [23]:
evaluate_model(model, val_loader, device, CLASS_NAMES)


              precision    recall  f1-score   support

       stand       0.00      0.00      0.00         0
         sit       1.00      0.75      0.85       177
  raise_hand       0.18      1.00      0.31        10

    accuracy                           0.76       187
   macro avg       0.39      0.58      0.39       187
weighted avg       0.96      0.76      0.83       187



## Save

In [24]:
save_model(model, "../outputs/models/pose_mlp.pth")

[INFO] Model saved to ../outputs/models/pose_mlp.pth
